In [6]:
import math
import operator

def createDataSet():
    dataSet = [[1, 1, 'maybe'],
              [1, 1, 'yes'],
              [1, 0, 'no'],
              [0, 1, 'no'],
              [0, 1, 'no']]
    labels = ['no surfacing', 'flippers']
    
    return dataSet, labels


def calcShannonEnt(dataSet):
    numEntries = len(dataSet)
    labelCounts = {}
    
    for featVec in dataSet:
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
    
    shannonEnt = 0.0
    
    for key in labelCounts:
        # using frequency(occurrence against total number) 
        # as an approximation for probability
        prob = float(labelCounts[key])/numEntries
        # prob  ~=  occcurence       / total instance number
        shannonEnt -= prob * math.log(prob, 2)
    
    return shannonEnt

def splitDataSet(dataSet, axis, value):
    """
    Axis is what nowadays so-called feature.
    """
    retDataSet = []
    
    for featVec in dataSet:
        # print "axis", axis
        # print "featVec[axis]", featVec[axis]
        # print "value", value
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]
            # print "featVec[:axis]", featVec[:axis]
            # print "featVec[axis+1:]", featVec[axis+1:]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    
    return retDataSet

def chooseBestFeatureToSplit(dataSet):
    # (feat1, feat2,...,tag), so the number of features
    # is len(dataset)-1 .The "1" is the tag column.
    numFeatures = len(dataSet[0])-1
    
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0
    bestFeature = -1
    
    for i in range(numFeatures):
        # featLis == (feat1, feat2,...,featn), without tag column
        # [1,1,1,0,0] [1,1,0,1,1]
        featList = [example[i] for example in dataSet]
        # {0,1}
        uniqueVals = set(featList)
        newEntropy = 0.0
        
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)
            
        infoGain = baseEntropy - newEntropy
        
        if(infoGain > bestInfoGain):
            bestInfoGain = infoGain
            bestFeature = i
    
    return bestFeature #, bestInfoGain

def majorityCnt(classList):
    classCount={}
    
    for vote in classList:        
        if vote not in classCount.keys(): classCount[vote] = 0
        classCount[vote] += 1
    
    sortedClassCount = sorted(classCount.iteritems(),
                             key=operator.itemgetter(1),
                             reverse=True)
    return sortedClassCount[0][0]

def createTree(dataSet, labels):
    classList = [example[-1] for example in dataSet]
    
    print classList
    print classList[0], len(classList)
    
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    
    if len(dataSet[0]) == 1:
        return majorityCnt(classList)
    
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    
    for value in uniqueVals:
        subLabels = labels[:]
                                       # recursive createTree
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet,
                                                              bestFeat,
                                                              value),
                                                 subLabels)
    
    return myTree

def main():
    # Train
    # Test
    myDat, labels = createDataSet()
    print myDat
    # print calcShannonEnt(myDat)
    # print splitDataSet(myDat, 0, 1)
    # print splitDataSet(myDat, 0, 0)
    myTree = createTree(myDat, labels)
    print myTree
    
if __name__ == "__main__":
    main()

[[1, 1, 'maybe'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
['maybe', 'yes', 'no', 'no', 'no']
maybe 5
['no', 'no']
no 2
['maybe', 'yes', 'no']
maybe 3
['no']
no 1
['maybe', 'yes']
maybe 2
{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'maybe'}}}}
